In [ ]:
!pip install scikit-learn==1.0
!pip install xgboost==1.4.2
!pip install catboost==0.26.1
!pip install pandas==1.3.3
!pip install radiant-mlhub==0.3.0
!pip install rasterio==1.2.8
!pip install numpy==1.21.2
!pip install pathlib==1.0.1
!pip install tqdm==4.62.3
!pip install joblib==1.0.1
!pip install matplotlib==3.4.3
!pip install Pillow==8.3.2
!pip install torch==1.9.1
!pip install plotly==5.3.1


In [ ]:
import warnings
warnings.filterwarnings('ignore')
# warnings.filterwarnings('RuntimeWarning')

from radiant_mlhub import Collection
import tarfile
import os
from pathlib import Path
import json
from tqdm import tqdm
from joblib import Parallel,delayed
import datetime
import rasterio
import numpy as np
import pandas as pd

import gc

gc.collect()

0

In [ ]:
competition_train_df = pd.read_csv('train_data_sentinel2.csv')

In [ ]:
def get_date_format(month,day):
    '''
        Structures the dates in a particular format
    '''
    if (str(month)=='nan') or (str(day)=='nan'):
        return 'nan'
    else:
        if month>=10:
            if day>=10:
                return f'month_{str(int(month))}_day_{str(int(day))}'
            else:
                return f'month_{str(int(month))}_day_0{str(int(day))}'
        else:
            if day>=10:
                return f'month_0{str(int(month))}_day_{str(int(day))}'
            else:
                return f'month_0{str(int(month))}_day_0{str(int(day))}'
    

In [ ]:
competition_train_df['month'] = pd.to_datetime(competition_train_df['datetime']).dt.month.values 
competition_train_df['day']   = pd.to_datetime(competition_train_df['datetime']).dt.day.values

competition_train_df['dates'] = competition_train_df.apply(lambda z: get_date_format(z['month'],z['day']),axis=1)

unique_dates = competition_train_df['dates'].unique()
unique_dates = np.array([z for z in unique_dates if 'nan' not in z])

print(f'Length of unique dates {len(unique_dates)}')

Length of unique dates 76


In [ ]:
date_dict = dict(zip(competition_train_df['datetime'].dropna().unique(),unique_dates))
date_dict = dict(sorted(date_dict.items(), key=lambda item: item[1]))
date_order_to_consider = np.array(list(date_dict.values()))
tile_ids_train = competition_train_df['tile_id'].unique()

In [ ]:
def get_bands(tile_date_times,tile_df,band,date_dict):
    '''
        Getting band dictionary with dates
    '''
    X_tile    = np.zeros((256 * 256, 76))
    X_tile[:] = np.nan
    
    for date_time in tile_date_times[:1]:
        
        
        source   = rasterio.open(tile_df[(tile_df['datetime']==date_time) & (tile_df['asset']==band)]['file_path'].values[0])

        ### Flattening the file to get a vector for the image 
        array    = np.expand_dims(source.read(1).flatten(), axis=1)

        ### Capturing the date at which we need to replace the vector
        val      = date_dict[date_time]

        ### index at which replacement is to be done
        indices  = np.where(date_order_to_consider==val)[0][0]            

        X_tile[:,indices]   = array.ravel()

    return X_tile

In [ ]:
def get_dataframe(data_dict_band,band,y,field_ids):
    
    X = np.array([values.tolist() for _,values in tqdm(data_dict_band.items())]).reshape(-1,76)
    colnames          = [band+'_'+z for z in date_order_to_consider]
    data              = pd.DataFrame(X,columns=colnames)
   
    data['field_id']  = field_ids
    
    mean_df           = data.groupby('field_id').mean().reset_index()
    low_df            = data.groupby('field_id').quantile(0.25).reset_index()
    up_df             = data.groupby('field_id').quantile(0.75).reset_index()
    med_df            = data.groupby('field_id').median().reset_index()
    
    
    return mean_df,low_df,up_df,med_df

In [ ]:
import xarray as xr
from rasterio.warp import transform

def convert_lat_lon(filename,crs):
    da = xr.open_rasterio(filename)
    x  = da['x']
    y  = da['y']
    ny, nx  = len(da['y']), len(da['x'])
    y, x    = np.meshgrid(da['y'], da['x'])
    
    lon,lat = transform(crs, {'init': 'EPSG:4326'},
                     x.flatten(), y.flatten())
    lon = np.asarray(lon).reshape((ny, nx))
    lat = np.asarray(lat).reshape((ny, nx))
    
    return lon,lat

In [ ]:
lb = 0 
ub = 200

bands_available = ['B01']

bigdf = []

for band in bands_available:

    for batch in tqdm(range(0,14)):
        print(f'Performing operations for batch {batch+1}/14 for band {band}')
        data_dict_band = {}
        

        count          = 1

        y              = np.empty((0, 1))


        field_ids      = np.empty((0, 1))
        encode_x_val   = np.empty((0, 1))
        encode_y_val   = np.empty((0, 1))

        for tile_id in tile_ids_train[lb+(batch*200):ub+(batch*200)]:
            if tile_id != '1951': # avoid using this specific tile for the Hackathon as it might have a missing file

                tile_df               = competition_train_df[competition_train_df['tile_id']==tile_id]

                field_id_src          = rasterio.open(tile_df[tile_df['asset']=='field_ids']['file_path'].values[0])

                field_id_array        = field_id_src.read(1)
                
                
                lon,lat               = convert_lat_lon(tile_df[tile_df['asset']=='field_ids']['file_path'].values[0],field_id_src.crs)
                
                
                encode_x = np.zeros((256,256))
                encode_y = np.zeros((256,256))

                for i in range(field_id_array.shape[0]):
                    for j in range(field_id_array.shape[1]):
                        if field_id_array[i,j]>0:
                            encode_x[i,j] = lat[i][j]
                            encode_y[i,j] = lon[i][j]
                            
                            
                tempdf = pd.DataFrame(columns = ['field_id','long','lat'])            
                tempdf['field_id'] = field_id_array.flatten()
                tempdf['long'] = encode_y.flatten()
                tempdf['lat'] = encode_x.flatten()
                
#                 print(tempdf.groupby(['field_id']).agg({'X':'median','Y':'median'}).reset_index().rename({'index':'field_id'}))
                gdf = tempdf.groupby(['field_id']).agg({'long':'median','lat':'median'}).reset_index().rename({'index':'field_id'})
                if len(bigdf)==0:
                    bigdf = gdf
                else:
                    bigdf = bigdf.append(gdf)
                    
                print(bigdf.shape)
                

                

                count                 = count+1
        gc.collect()        

#         mean_df,low_df,up_df,med_df = get_dataframe(data_dict_band,band,y,field_ids)
        
        batchid           = int(batch)+1

#         mean_df.to_csv(f'train_position.csv',index=False)
        
        

#         del field_ids,data_dict_band,label_src,field_id_src,label_array,field_id_array,y,mean_df,med_df,up_df,low_df
#         del tile_df,tile_date_times
#         gc.collect()
        gc.collect()
        gc.collect()

  0%|          | 0/14 [00:00<?, ?it/s]

Performing operations for batch 1/14 for band B01
(13, 3)
(28, 3)
(61, 3)
(110, 3)
(127, 3)
(135, 3)
(156, 3)
(161, 3)
(182, 3)
(303, 3)
(308, 3)
(333, 3)
(381, 3)
(419, 3)
(424, 3)
(454, 3)
(456, 3)
(459, 3)
(491, 3)
(500, 3)
(512, 3)
(522, 3)
(787, 3)
(822, 3)
(858, 3)
(875, 3)
(927, 3)
(948, 3)
(972, 3)
(1020, 3)
(1174, 3)
(1181, 3)
(1194, 3)
(1217, 3)
(1234, 3)
(1249, 3)
(1254, 3)
(1291, 3)
(1324, 3)
(1345, 3)
(1347, 3)
(1455, 3)
(1627, 3)
(1643, 3)
(1669, 3)
(1683, 3)
(1710, 3)
(1746, 3)
(1775, 3)
(1854, 3)
(1876, 3)
(1891, 3)
(1902, 3)
(1923, 3)
(1939, 3)
(1942, 3)
(1994, 3)
(2028, 3)
(2051, 3)
(2065, 3)
(2071, 3)
(2119, 3)
(2159, 3)
(2220, 3)
(2239, 3)
(2295, 3)
(2348, 3)
(2358, 3)
(2360, 3)
(2365, 3)
(2407, 3)
(2409, 3)
(2451, 3)
(2461, 3)
(2483, 3)
(2489, 3)
(2533, 3)
(2538, 3)
(2552, 3)
(2554, 3)
(2567, 3)
(2586, 3)
(2590, 3)
(2698, 3)
(2721, 3)
(2727, 3)
(2879, 3)
(2895, 3)
(2904, 3)
(2946, 3)
(3032, 3)
(3057, 3)
(3060, 3)
(3112, 3)
(3218, 3)
(3276, 3)
(3413, 3)
(3453, 3)
(3

  7%|▋         | 1/14 [00:45<09:45, 45.07s/it]

(6267, 3)
Performing operations for batch 2/14 for band B01
(6274, 3)
(6312, 3)
(6319, 3)
(6390, 3)
(6422, 3)
(6465, 3)
(6500, 3)
(6506, 3)
(6509, 3)
(6520, 3)
(6533, 3)
(6547, 3)
(6551, 3)
(6683, 3)
(6706, 3)
(6711, 3)
(6760, 3)
(6789, 3)
(6806, 3)
(6832, 3)
(6912, 3)
(6926, 3)
(6929, 3)
(6996, 3)
(7015, 3)
(7027, 3)
(7032, 3)
(7065, 3)
(7437, 3)
(7475, 3)
(7487, 3)
(7745, 3)
(7764, 3)
(7804, 3)
(7812, 3)
(7849, 3)
(7852, 3)
(7863, 3)
(7897, 3)
(8034, 3)
(8057, 3)
(8060, 3)
(8119, 3)
(8156, 3)
(8163, 3)
(8195, 3)
(8213, 3)
(8258, 3)
(8284, 3)
(8364, 3)
(8391, 3)
(8441, 3)
(8480, 3)
(8484, 3)
(8511, 3)
(8514, 3)
(8516, 3)
(8547, 3)
(8558, 3)
(8583, 3)
(8591, 3)
(8600, 3)
(8604, 3)
(8607, 3)
(8652, 3)
(8660, 3)
(8673, 3)
(8696, 3)
(8814, 3)
(8822, 3)
(8826, 3)
(8838, 3)
(8862, 3)
(8925, 3)
(8935, 3)
(8952, 3)
(8954, 3)
(8993, 3)
(9017, 3)
(9211, 3)
(9221, 3)
(9244, 3)
(9259, 3)
(9266, 3)
(9280, 3)
(9307, 3)
(9363, 3)
(9367, 3)
(9406, 3)
(9445, 3)
(9471, 3)
(9480, 3)
(9491, 3)
(9554, 3)


 14%|█▍        | 2/14 [01:29<08:56, 44.70s/it]

...

(80458, 3)
Performing operations for batch 13/14 for band B01
(80460, 3)
(80479, 3)
(80529, 3)
(80545, 3)
(80622, 3)
(80803, 3)
(80861, 3)
(80871, 3)
(80896, 3)
(80901, 3)
(80967, 3)
(81003, 3)
(81171, 3)
(81183, 3)
(81246, 3)
(81263, 3)
(81283, 3)
(81287, 3)
(81299, 3)
(81310, 3)
(81490, 3)
(81492, 3)
(81499, 3)
(81540, 3)
(81544, 3)
(81550, 3)
(81578, 3)
(81599, 3)
(81601, 3)
(81652, 3)
(81719, 3)
(81779, 3)
(81812, 3)
(81814, 3)
(81818, 3)
(81828, 3)
(81851, 3)
(81990, 3)
(82086, 3)
(82137, 3)
(82294, 3)
(82302, 3)
(82356, 3)
(82366, 3)
(82368, 3)
(82436, 3)
(82439, 3)
(82473, 3)
(82516, 3)
(82565, 3)
(82575, 3)
(82632, 3)
(82643, 3)
(82694, 3)
(82705, 3)
(82711, 3)
(82733, 3)
(82735, 3)
(82776, 3)
(82813, 3)
(82850, 3)
(82984, 3)
(83019, 3)
(83024, 3)
(83027, 3)
(83048, 3)
(83050, 3)
(83067, 3)
(83090, 3)
(83123, 3)
(83203, 3)
(83230, 3)
(83238, 3)
(83277, 3)
(83282, 3)
(83299, 3)
(83322, 3)
(83360, 3)
(83583, 3)
(83604, 3)
(83667, 3)
(83710, 3)
(83757, 3)
(83763, 3)
(83789, 3)
(83

 93%|█████████▎| 13/14 [09:40<00:44, 44.58s/it]

(88214, 3)
Performing operations for batch 14/14 for band B01
(88219, 3)
(88268, 3)
(88325, 3)
(88345, 3)
(88348, 3)
(88372, 3)
(88396, 3)
(88403, 3)
(88416, 3)
(88433, 3)
(88440, 3)
(88446, 3)
(88493, 3)
(88502, 3)
(88508, 3)
(88525, 3)
(88557, 3)
(88562, 3)
(88586, 3)
(88633, 3)
(88723, 3)
(88764, 3)
(88809, 3)
(88874, 3)
(88895, 3)
(88897, 3)
(88967, 3)
(88973, 3)
(89034, 3)
(89046, 3)
(89048, 3)
(89050, 3)
(89079, 3)
(89120, 3)
(89188, 3)
(89208, 3)
(89249, 3)
(89301, 3)
(89321, 3)
(89406, 3)
(89409, 3)
(89438, 3)
(89467, 3)
(89489, 3)
(89503, 3)
(89522, 3)
(89705, 3)
(89754, 3)
(89759, 3)


100%|██████████| 14/14 [09:51<00:00, 42.27s/it]

(89763, 3)


In [ ]:
train_coordinates = bigdf[bigdf['field_id']!=0]

In [ ]:
train_coordinates.to_csv('train_coordinates_lat_lon.csv',index=False)

In [ ]:
train_coordinates

,field_id,long,lat
1,3020,18.574318,-33.570163
2,7478,18.594046,-33.587416
3,15902,18.576833,-33.576354
4,38846,18.577147,-33.591117
5,42856,18.595662,-33.585112
...,...,...,...
3,93418,18.305301,-32.569756
4,97927,18.309672,-32.569369
1,73261,18.805962,-31.880199
2,92107,18.801426,-31.879400
